In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import re
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import re
import pickle

In [ ]:
with open(<'FILE NAME'>, 'rb') as f:
    prod_processed = pickle.load(f)
prod_processed = prod_processed.reset_index(drop=True)

### Data Processing

In [ ]:
prod_processed = prod_processed.drop_duplicates(keep='first')
prod_processed.shape

In [ ]:
first_drop = ['Second_Zertifikat','Second_FSC®-Code','Zertifikat','Bestellung', 'Profil',
               'Aufbauhöhe', 'Kunstoffgitter','Maßanfertigung', 'Verschraubung', 'Füße', 'Lamellen', 'Verleimung',
               'Zertifiikat', 'Längsseite', 'Second_Holzlamellen', 'Nachhaltigkeit', 'Gesamthöhe','Tiefe', 'Höhe',
               'Second_Unterkonstruktion','Stärke Unterkonstruktion', 'Bedarf Verbindungsadapter', 'Handgriffe', 'Detail', 
                'Variante','Maße','Holzstärke','Längsseiten','Mindestlänge','Innenmaß Sitzfläche', 'Second_Durchmesser', 'Durchmesser',
               'Befestigung', 'Gewicht', 'Klappmechanismus', 'Breite Sitzfläche',
               'Second_Gewicht', 'Verpackungseinheit', 'Höhe Sitzfläche',
               'Höhe Armlehnen', 'Kanten', 'Längsseiten', 'Second_Menge',
               'Mindestlängen', 'Mindestlänge','Second_Behandlung', 'Behandlung', 'Verarbeitung', 'Herkunft',
               'Länge Element', 'Breite Dielen', 'Breite Element','', 'Second_Marke', 'Second_Oberfläche', 'Second_Qualität', 
               'Second_Breite', 'Second_Länge', 'Second_Stärke','Menge je KG bei 120mm Breite', 'Menge je kg',
               'Menge je KG bei 90mm Breite', 'Holzlamellen', 'Lebensdauer',
               'Menge je kg bei 50mm Breite', 'Menge je kg bei 95mm Breite',
               'Mögliche Fehler', 'Breite Holzmuster', 'Länge Holzmuster','Produktart']

In [ ]:
prod_processed['Marke'] = prod_processed['Marke'].fillna(prod_processed['Second_Marke'])
prod_processed['Oberfläche'] = prod_processed['Oberfläche'].fillna(prod_processed['Second_Oberfläche'])
prod_processed['Qualität'] = prod_processed['Qualität'].fillna(prod_processed['Second_Qualität'])
prod_processed['Breite'] = prod_processed['Breite'].fillna(prod_processed['Second_Breite']).fillna(prod_processed['Breite Holzmuster'])
prod_processed['Länge'] = prod_processed['Länge'].fillna(prod_processed['Second_Länge']).fillna(prod_processed['Länge Holzmuster'])
prod_processed['Stärke'] = prod_processed['Stärke'].fillna(prod_processed['Second_Stärke'])
prod_processed['Option_Grosse'] = prod_processed['Option_Grosse'].fillna(prod_processed['Maße'])
prod_processed['Oberfläche'] = prod_processed['Oberfläche']  + ', ' + prod_processed['Behandlung']

In [ ]:
prod_processed = prod_processed.drop(columns = first_drop).copy()
prod_processed.shape

In [ ]:
rename_cols = {'Second_Produktart':'Product Type', 
               'Second_Holzart':'Sec_Wood_Type',
               'Second_Herkunft':'Origin', 
               'Second_Farbe':'Color',
               'Option_Grosse':'Option_Size',
               'Second_Größe':'Other Dimensions',
               'Holzart':'Wood type',
               'Option':'Option_Size',
               'Oberfläche_NEW':'Surfase',
               'Qualität':'Quality',
               'Breite':'Possible Width',
               'Länge':'Possible Length',
               'Main Header':'Product Title',
               'Stärke':'Thickness',
               'Trocknung':'Drying Method'}
prod_processed = prod_processed.rename(columns=rename_cols)

In [ ]:
(prod_processed.isna().sum()/prod_processed.shape[0]*100).sort_values(ascending=False)

### Cleaning data

In [ ]:
clean_cols = ['Product Title', 'Wood type', 'Product Type', 'Surfase','Quality',
              'Drying Method', 'Main Delivery','Origin','Color']

cols = ['Product Title', 'Wood type', 'Product Type', 'Surfase','Quality','Drying Method',
        'Index','Main Price', 'Possible Length','Possible Width','Thickness', 'Other Dimensions', 'Option_Size', 
        'Option_Width','Option_Length', 'Option_Price', 'Main Delivery','Origin','Color','SKU','URL']

In [ ]:
for col in clean_cols:
    prod_processed[col] = prod_processed[col].str.replace(r'\n|^ | $|Voraussichtliche Lieferung: ','')

In [ ]:
prod_processed['Option_Price'] = prod_processed['Option_Price'].fillna(prod_processed['Main Price'])

In [ ]:
prod_processed[(prod_processed['Option_Size'].isna()) & 
               (prod_processed['Option_Width'].isna()) &
               (prod_processed['Option_Length'].isna())][cols].shape

### Clean Optional/ Possible Length/Width

In [ ]:
prod_processed['Option_Size_Length'] = prod_processed['Option_Size'].str.split('x', expand=True)[0] + ' cm'
prod_processed['Option_Size_Width'] = prod_processed['Option_Size'].str.split('x', expand=True)[1]

In [ ]:
prod_processed['Option_Width'] = prod_processed['Option_Width'].fillna(prod_processed['Option_Size_Width'])
prod_processed['Option_Length'] = prod_processed['Option_Length'].fillna(prod_processed['Option_Size_Length'])

prod_processed['Option_Width'] = prod_processed['Option_Width'].fillna(prod_processed['Possible Width'])
prod_processed['Option_Length'] = prod_processed['Option_Length'].fillna(prod_processed['Possible Length'])

In [ ]:
prod_processed[['Option_Width', 'Option_Length']].isna().sum()

#### Unifying Width dimentions

In [ ]:
prod_processed['Option_Width_New'] = prod_processed['Option_Width'].str.split('bis ', expand=True)[1]
prod_processed['Option_Width_New'] = prod_processed['Option_Width_New'].fillna(prod_processed['Option_Width'])
prod_processed['Option_Width_New'] = prod_processed['Option_Width_New'].fillna(0).astype(str).str.replace(r'\n|^ | $|ca\. | \(\+\/\- 1cm\)| ','')
prod_processed['Option_Width_New'] = prod_processed['Option_Width_New'].str.replace(r'\,','.')

In [ ]:
prod_processed['Width_Mult'] = np.nan
def get_mult(row):

    if 'mm' in row['Option_Width_New']:
        row['Width_Mult'] = 1
    elif 'cm' in row['Option_Width_New']:
        row['Width_Mult'] = 10
    else:
        row['Width_Mult'] = 10
    return row

prod_processed = prod_processed.apply(get_mult, axis=1)

In [ ]:
prod_processed['Option_Width_Clean'] = prod_processed['Option_Width_New'].str.extract(r'(.+(?=\w\w))', expand=True)[0]

In [ ]:
clean_width = prod_processed['Option_Width_New'].str.replace(r'[a-z]+','').astype(float)
prod_processed['Option_Width_mm'] = clean_width*prod_processed['Width_Mult'].astype(float)

#### Unifying Length dimentions

In [ ]:
prod_processed['Option_Length_New'] = prod_processed['Option_Length'].str.split('bis ', expand=True)[1]
prod_processed['Option_Length_New'] = prod_processed['Option_Length_New'].fillna(prod_processed['Option_Length'])
prod_processed['Option_Length_New'] = prod_processed['Option_Length_New'].fillna(0).astype(str).str.replace(r'\n|^ | $|ca\. | \(\+\/\- 1cm\)| ','')
prod_processed['Option_Length_New'] = prod_processed['Option_Length_New'].str.replace(r'\,','.')
prod_processed['Option_Length_New'] = prod_processed['Option_Length_New'].str.replace(r'(\.fallendeLängen|\.geteilteHolzlamelle|zu)','')

In [ ]:
prod_processed['Length_Mult'] = np.nan
def get_mult(row):

    if 'mm' in row['Option_Length_New']:
        row['Length_Mult'] = 1
    elif 'cm' in row['Option_Length_New']:
        row['Length_Mult'] = 10
    else:
        row['Length_Mult'] = 10
    return row

prod_processed = prod_processed.apply(get_mult, axis=1)

In [ ]:
prod_processed['Option_Length_Clean'] = prod_processed['Option_Length_New'].str.extract(r'(.+(?=\w\w))', expand=True)[0]

In [ ]:
clean_length = prod_processed['Option_Length_New'].str.replace(r'[a-z]+','').astype(float)
prod_processed['Option_Length_mm'] = clean_length*prod_processed['Length_Mult'].astype(float)

### Cleaning Wood type

In [ ]:
prod_processed['Wood type'] = prod_processed['Wood type'].str.replace(r' \(.+\)','').str.replace(r'TEak','Teak')

### Cleaning Price

In [ ]:
prod_processed['Option_Price'] = prod_processed['Option_Price'].str.replace(r'( €$)','').str.replace(r'(.+(?=€ ))','').str.replace(r'(^€|^ )','')
prod_processed['Option_Price'] = prod_processed['Option_Price'].str.replace(r'(\,)','.')

In [ ]:
prod_processed['Option_Price'] = prod_processed['Option_Price'].apply(lambda x: np.nan if x == ' ' or x == '' else x)
prod_processed['Option_Price'] = prod_processed['Option_Price'].astype(float)
prod_processed['Option_Price'].isna().sum()

In [ ]:
for col in prod_processed.columns:
    prod_processed[col] = prod_processed[col].apply(lambda x: np.nan if x==-999 or x == '' or x == ' ' else x)

# To Google Sheets

In [ ]:
import json
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from df2gspread import df2gspread as d2g
scope = ['https://spreadsheets.google.com/feeds'] 
credentials = ServiceAccountCredentials.from_json_keyfile_name('/Users/macbook/Downloads/gs-credentials.json', scope) 
gc = gspread.authorize(credentials)
spreadsheet_key = '1iDGux_PxTPSIkxnljeEhMifJo5jAXDhxOXLB8zlrOKg'

In [ ]:
cols = ['Product Title', 'Wood type', 'Product Type', 'Surfase','Quality','Drying Method','Thickness', 
        'Option_Width_mm','Option_Length_mm', 'Option_Price', 'Main Delivery','Origin','Color','SKU','URL']

In [ ]:
to_sheets = prod_processed[cols].fillna('-').copy()
to_sheets.shape

In [ ]:
to_sheets = to_sheets.rename(columns={'Option_Width_mm':'Width, mm', 
                                      'Option_Length_mm':'Length, mm',
                                      'Option_Price':'Price, EUR'})

In [ ]:
d2g.upload(to_sheets,
           spreadsheet_key,
           'Products Update',
           credentials=credentials,
           col_names=True,
           row_names=False,
           start_cell = 'A1',
           clean=True)